# Phenotype Clustering

In [8]:
import glob
import itertools as it 
import os
import sys
from textwrap import wrap

from adjustText import adjust_text
import dynamicTreeCut
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist
import seaborn as sns
sns.set(rc={'font.sans-serif':'Helvetica'})
sns.set(style='whitegrid', rc={'font.sans-serif':'Helvetica'})

from IPython.display import display, HTML

%matplotlib inline

In [5]:
dynamicTreeCut.cutreeHybrid?

In [65]:
corrs = pd.read_table('../../h2-estimation/private_output/ukb_stan/combined/vtu_opt_100_0.5.tsv.gz')
corrs = corrs[(corrs['p1_category'].isnull() == False) & 
              (corrs['p2_category'].isnull() == False)]
corrs_p = corrs.pivot(index='p1_code', columns='p2_code', values='omegacor21')
for pheno in set(corrs_p.index) - set(corrs_p.columns):
    corrs_p[pheno] = np.nan
corrs_p = corrs_p.T
for pheno in set(corrs_p.index) - set(corrs_p.columns):
    corrs_p[pheno] = np.nan
corrs_p = corrs_p.T
corrs_p = corrs_p.loc[corrs_p.columns]
for p in corrs_p.index:
    corrs_p.loc[p, p] = 1
corrs_p = pd.DataFrame(np.triu(corrs_p) + np.triu(corrs_p,1).T, 
                       index=corrs_p.index, columns=corrs_p.columns)

In [77]:
corrs_p = corrs_p.replace(np.nan, 0)

In [78]:
link = linkage(corrs_p, "average")

In [79]:
clusters = dynamicTreeCut.cutreeHybrid(link, distances)

('..cutHeight not given, setting it to', 32.765573934241289, ' ===>  99% of the (truncated) height range in dendro.')


ValueError: cannot convert float NaN to integer

In [10]:
d = np.transpose(np.arange(1,10001).reshape(100,100))
distances = pdist(d, "euclidean")
link = linkage(distances, "average")
clusters = dynamicTreeCut.cutreeHybrid(link, distances)
#..cutHeight not given, setting it to 495.1  ===>  99% of the (truncated) height range in dendro.
#..done.
clusters["labels"]

('..cutHeight not given, setting it to', 495.10000000000002, ' ===>  99% of the (truncated) height range in dendro.')
..done.


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])